# Algoritmo 3 - Associação de Regras

Vamos usar o Apriori para encontrar associações entre os desportos praticados pelos atletas olímpicos.

## Dataset:
Transferir em: https://www.kaggle.com/datasets/stefanoleone992/ea-sports-fc-24-complete-player-dataset

Introduzir em `Data/EA_FC`

## Introdução
Este notebook aplica mineração de regras de associação para estimar a probabilidade de vencer uma medalha num desporto sabendo que já foi conquistada outra medalha noutro desporto. Partimos da consolidação dos ficheiros Summer e Winter, enriquecemos os registos com dados demográficos e, no fim, extraímos padrões frequentes com o algoritmo Apriori.


In [3]:
print("Hello World!")

%pip install --upgrade pip
%pip install numpy
%pip install matplotlib
%pip install pandas
%pip install mlxtend


Hello World!
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Preparação do ambiente
As linhas seguintes asseguram que todas as bibliotecas necessárias (pandas, numpy, matplotlib e mlxtend) estão instaladas e atualizadas. Executa-as apenas quando configurares o notebook pela primeira vez ou sempre que mudares de ambiente.

# Importar Ficheiro CSV
Carregamos os ficheiros `winter.csv` e `summer.csv`, adicionamos a coluna `Season` para distinguir as observações e fazemos o `concat` para ter um único dataset. Em seguida, renomeamos `Country` para `Country_Code` para alinhar com o dicionário e juntamos informação demográfica como população e PIB per capita.

In [4]:
import pandas as pd

# --- Carregar ---
dados_inverno = pd.read_csv("Data/Olympics/winter.csv")
dados_inverno["Season"] = "Winter"

dados_verao = pd.read_csv("Data/Olympics/summer.csv")
dados_verao["Season"] = "Summer"

# --- Limpeza antes de concatenar ---
COLUNAS_TEXTO = ["City", "Sport", "Discipline", "Athlete", "Country", "Gender", "Event", "Medal"]

def limpar_df_temporada(df: pd.DataFrame) -> pd.DataFrame:
    df = df.drop_duplicates().copy()  # remove duplicados exatos
    for coluna in COLUNAS_TEXTO:
        df[coluna] = df[coluna].astype("string").str.strip()
    return df

dados_inverno = limpar_df_temporada(dados_inverno)
dados_verao = limpar_df_temporada(dados_verao)

# --- Unificar ---
dados_consolidados = pd.concat([dados_inverno, dados_verao], ignore_index=True)

# Harmonizar código do país
dados_consolidados = dados_consolidados.rename(columns={"Country": "Country_Code"})

# --- Limpeza que precisa do dataset todo ---

# 1) Remover linhas "Pending"
dados_consolidados = dados_consolidados[dados_consolidados["Athlete"].ne("Pending")].copy()

# 2) Preencher Country_Code em falta pelo país mais frequente do mesmo atleta, no caso do KUDUKHOV, Besik em 2012
modo_pais = (
    dados_consolidados.dropna(subset=["Country_Code"])
              .groupby("Athlete")["Country_Code"]
              .agg(lambda x: x.mode().iat[0] if not x.mode().empty else pd.NA)
)
mascara_em_falta = dados_consolidados["Country_Code"].isna()
dados_consolidados.loc[mascara_em_falta, "Country_Code"] = (
    dados_consolidados.loc[mascara_em_falta, "Athlete"].map(modo_pais)
)

# 3) reduzir inflacionamento por equipas:
#    garante que cada (país, ano, season, sport, event, medal) conta 1 vez
dados_consolidados = dados_consolidados.drop_duplicates(
    subset=["Year", "Season", "Country_Code", "Sport", "Event", "Medal"]
)

# 4) Normalização semântica (para consistência)
DISCIPLINE_MAP = {
    "Wrestling Free.": "Wrestling Freestyle",
    "Wrestling Gre-R": "Wrestling Greco-Roman",
    "Artistic G.": "Artistic Gymnastics",
    "Rhythmic G.": "Rhythmic Gymnastics",
    "Modern Pentath.": "Modern Pentathlon",
    "Synchronized S.": "Synchronized Swimming",
    "Beach volley.": "Beach Volleyball",
}
SPORT_MAP = {"Canoe": "Canoe / Kayak"}

dados_consolidados["Discipline"] = dados_consolidados["Discipline"].replace(DISCIPLINE_MAP)
dados_consolidados["Sport"] = dados_consolidados["Sport"].replace(SPORT_MAP)

# --- Dicionário: limpar e fazer merge com a ponte de códigos ---
dicionario = pd.read_csv("Data/Olympics/dictionary.csv").copy()
dicionario["Country"] = dicionario["Country"].astype("string").str.replace("*", "", regex=False).str.strip()

# Alguns códigos do dataset olímpico não existem no dicionário com o mesmo nome
NOC_PARA_DIC = {
    "ROU": "ROM",  # Romania
    "SGP": "SIN",  # Singapore
    "TTO": "TRI",  # Trinidad e Tobago
    "SRB": "SCG",  # (Serbia está como SCG)
}
dados_consolidados["codigo_dic"] = dados_consolidados["Country_Code"].replace(NOC_PARA_DIC)

olimpiadas = dados_consolidados.merge(
    dicionario[["Code", "Country", "Population", "GDP per Capita"]],
    left_on="codigo_dic",
    right_on="Code",
    how="left"
)

# se juntar falhar, isto evita perder o país no groupby
olimpiadas["pais_final"] = olimpiadas["Country"].fillna(olimpiadas["Country_Code"])


## Construir transações
Criamos a variável `SportMedal` para combinar desporto e medalha e agregamos por país/ano/temporada para obter o conjunto de itens por participação olímpica. Apenas mantemos transações com pelo menos um item antes de aplicar o `TransactionEncoder`.

In [5]:
from mlxtend.preprocessing import TransactionEncoder

def construir_cesto(df: pd.DataFrame) -> tuple[pd.DataFrame, pd.DataFrame]:
    # Criar itemsets por país/ano/temporada
    df_medalhas = df[df["Medal"].isin(["Gold", "Silver", "Bronze"])].copy()

    transacoes_df = (
        df_medalhas
        .groupby(["pais_final", "Year", "Season"])["Sport"]
        .unique()
        .reset_index()
    )

    transacoes_df["num_itens"] = transacoes_df["Sport"].apply(len)
    transacoes_df = transacoes_df[transacoes_df["num_itens"] >= 1].reset_index(drop=True)

    transacoes = transacoes_df["Sport"].apply(list).tolist()

    te = TransactionEncoder()
    te_matriz = te.fit(transacoes).transform(transacoes)
    cesto = pd.DataFrame(te_matriz, columns=te.columns_)

    return cesto, transacoes_df

olimpiadas_verao = olimpiadas[olimpiadas["Season"] == "Summer"].copy()
olimpiadas_inverno = olimpiadas[olimpiadas["Season"] == "Winter"].copy()

cesto_verao, transacoes_verao = construir_cesto(olimpiadas_verao)
cesto_inverno, transacoes_inverno = construir_cesto(olimpiadas_inverno)

cestos = {
    "Verao": cesto_verao,
    "Inverno": cesto_inverno,
}


## Descobrir regras de associação
Aplicamos o `apriori` para gerar `frequent_itemsets` com suporte mínimo de 2% e, em seguida, calculamos regras com confiança mínima de 0.5. Filtramos apenas regras multi-antecedente (2 → 1) com suporte relevante e `lift` > 1.2 para destacar relações entre desporto e medalha.


In [6]:
from mlxtend.frequent_patterns import apriori, association_rules
from IPython.display import display

SUPORTE_MIN = 0.02
CONFIANCA_MIN = 0.5
LIFT_MIN_MULTI = 1.2

# Controlar a complexidade, vamos por 2 antecedentes e 1 consequente
def extrair_regras(cesto: pd.DataFrame) -> pd.DataFrame:
    itemsets_frequentes = apriori(
        cesto,
        min_support=SUPORTE_MIN,
        use_colnames=True,
        max_len=3
    )

    regras = association_rules(itemsets_frequentes, metric="confidence", min_threshold=CONFIANCA_MIN)

    # calcular comprimentos (uma vez so)
    regras["tamanho_antecedente"] = regras["antecedents"].apply(len)
    regras["tamanho_consequente"] = regras["consequents"].apply(len)

    regras_multi_antecedente = (
        regras[
            (regras["tamanho_antecedente"] >= 2) &
            (regras["tamanho_consequente"] == 1) &
            (regras["support"] >= SUPORTE_MIN) &
            (regras["lift"] >= LIFT_MIN_MULTI)
        ]
        .sort_values(["lift", "confidence", "support"], ascending=False)
    )

    # antecedente no max 2
    regras_multi_antecedente = regras_multi_antecedente[regras_multi_antecedente["tamanho_antecedente"] <= 2]

    return regras_multi_antecedente

regras_multi_por_temporada = {}

for temporada, cesto in cestos.items():
    regras_multi_antecedente = extrair_regras(cesto)
    regras_multi_por_temporada[temporada] = regras_multi_antecedente

    print(f"=== {temporada}: Regras multi-antecedente (2+ -> 1) ===")
    display(regras_multi_antecedente)


=== Verao: Regras multi-antecedente (2+ -> 1) ===


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski,tamanho_antecedente,tamanho_consequente
969,"(Basketball, Shooting)",(Volleyball),0.037165,0.062230,0.020743,0.558140,8.968992,1.0,0.018431,2.122322,0.922801,0.263736,0.528818,0.445736,2,1
967,"(Volleyball, Shooting)",(Basketball),0.039758,0.060501,0.020743,0.521739,8.623602,1.0,0.018338,1.964406,0.920642,0.260870,0.490940,0.432298,2,1
637,"(Athletics, Basketball)",(Volleyball),0.043215,0.062230,0.021608,0.500000,8.034722,1.0,0.018918,1.875540,0.915086,0.257732,0.466820,0.423611,2,1
1635,"(Volleyball, Sailing)",(Judo),0.027658,0.178911,0.024201,0.875000,4.890700,1.0,0.019252,6.568712,0.818159,0.132701,0.847763,0.505133,2,1
1445,"(Hockey, Rowing)",(Equestrian),0.031979,0.152982,0.023336,0.729730,4.770041,1.0,0.018444,3.133967,0.816468,0.144385,0.680916,0.441136,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
673,"(Taekwondo, Boxing)",(Athletics),0.029386,0.528090,0.020743,0.705882,1.336671,1.0,0.005225,1.604494,0.259498,0.038647,0.376751,0.372581,2,1
895,"(Wrestling, Taekwondo)",(Athletics),0.033708,0.528090,0.023336,0.692308,1.310966,1.0,0.005535,1.533708,0.245478,0.043339,0.347985,0.368249,2,1
904,"(Wrestling, Weightlifting)",(Athletics),0.140017,0.528090,0.095938,0.685185,1.297478,1.0,0.021996,1.499009,0.266603,0.167674,0.332892,0.433427,2,1
846,"(Wrestling, Judo)",(Athletics),0.100259,0.528090,0.068280,0.681034,1.289618,1.0,0.015334,1.479502,0.249602,0.121914,0.324097,0.405165,2,1


=== Inverno: Regras multi-antecedente (2+ -> 1) ===


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski,tamanho_antecedente,tamanho_consequente
15,"(Biathlon, Bobsleigh)",(Luge),0.050132,0.139842,0.042216,0.842105,6.021847,1.0,0.035206,5.447669,0.877951,0.285714,0.816435,0.571996,2,1
34,"(Skiing, Bobsleigh)",(Luge),0.137203,0.139842,0.068602,0.500000,3.575472,1.0,0.049415,1.720317,0.834862,0.329114,0.418712,0.495283,2,1
30,"(Skating, Luge)",(Bobsleigh),0.087071,0.216359,0.058047,0.666667,3.081301,1.0,0.039209,2.350923,0.739884,0.236559,0.574635,0.467480,2,1
13,"(Luge, Biathlon)",(Bobsleigh),0.065963,0.216359,0.042216,0.640000,2.958049,1.0,0.027945,2.176781,0.708686,0.175824,0.540606,0.417561,2,1
32,"(Luge, Skiing)",(Bobsleigh),0.121372,0.216359,0.068602,0.565217,2.612407,1.0,0.042342,1.802375,0.702472,0.254902,0.445176,0.441145,2,1
20,"(Skating, Luge)",(Biathlon),0.087071,0.226913,0.050132,0.575758,2.537350,1.0,0.030374,1.822277,0.663675,0.190000,0.451236,0.398344,2,1
14,"(Luge, Bobsleigh)",(Biathlon),0.079156,0.226913,0.042216,0.533333,2.350388,1.0,0.024255,1.656615,0.623926,0.160000,0.396359,0.359690,2,1
23,"(Luge, Skiing)",(Biathlon),0.121372,0.226913,0.060686,0.500000,2.203488,1.0,0.033145,1.546174,0.621622,0.211009,0.353242,0.383721,2,1
19,"(Biathlon, Ice Hockey)",(Skiing),0.044855,0.693931,0.044855,1.000000,1.441065,1.0,0.013729,inf,0.320442,0.064639,1.000000,0.532319,2,1
28,"(Ice Hockey, Bobsleigh)",(Skating),0.052770,0.591029,0.044855,0.850000,1.438170,1.0,0.013666,2.726473,0.321645,0.074890,0.633226,0.462946,2,1


## Interpretar resultados
Os dataframes em `regras_multi_por_temporada` contêm as associações mais fortes (2 → 1) entre desportos ordenadas por `lift`. Analisando estas tabelas é possível identificar quais desportos têm maior probabilidade de ganhar medalha sabendo que já foi conquistada outra noutro desporto.


## Conclusão
O notebook consolidado permitiu caracterizar o panorama olímpico ao nível de país, edição e temporada, revelando padrões consistentes de medalhas entre desportos. As regras com maior lift ajudam a estimar a probabilidade de vencer uma medalha num desporto sabendo que já foi conquistada outra noutro desporto, orientando a análise para relações mais fortes e úteis no relatório final. Ajusta os limiares de suporte/confiança conforme necessário para aprofundar segmentos específicos ou testar hipóteses adicionais.
